In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType
from pyspark.sql.functions import year, month, concat_ws, broadcast, sum, col, avg, count

In [2]:
#Create a Spark Session
#Define Schemas for our dataFrame based on the csv file we have stored in Google Cloud Storage Bucket.

spark = SparkSession.builder\
        .appName("Retail DataSet")\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/17 18:26:21 INFO SparkEnv: Registering MapOutputTracker
25/05/17 18:26:21 INFO SparkEnv: Registering BlockManagerMaster
25/05/17 18:26:21 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/05/17 18:26:21 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
#Reading the data from Cloud Storage Bucket and creating dataFrame

In [3]:
customers_df = spark.read\
        .option("header", True)\
        .option("mode", "PERMISSIVE")\
        .option("badRecordsPath", "gs://retail_pysparkdata/bad_data")\
        .option("nullValue", "")\
        .option("inferSchema", True)\
        .csv("gs://retail_pysparkdata/customers.csv")

transactions_df = spark.read\
            .option("header", True)\
            .option("mode", "PERMISSIVE")\
            .option("badRecordsPath", "gs://retail_pysparkdata/bad_data")\
            .option("nullValue", "")\
            .option("inferSchema", True)\
            .csv("gs://retail_pysparkdata/transactions.csv")

In [5]:
#Clean data - nulls & remove duplicates

In [4]:
customers_df_clean = customers_df.dropna(subset = ["customer_id"]).dropDuplicates(["customer_id"])

In [5]:
transactions_df_clean = transactions_df.dropna(subset = ["customer_id","transaction_id","quantity","price"]).dropDuplicates(["customer_id"])

In [6]:
#Add transaction year/month and full address new column

transactions_df_enriched = transactions_df_clean.withColumn("transaction_year", year("transaction_date"))\
                            .withColumn("transaction_month", month("transaction_date"))

customers_df_enriched = customers_df_clean.withColumn("full_address", concat_ws(", ", "street_address", "city", "state", "zip_code"))


In [9]:
customers_count = transactions_df_enriched.groupBy("customer_id").agg(count(col("transaction_id")).alias("count_cust"))

In [13]:
customers_count = transactions_df_enriched.groupBy("customer_id").agg(count(col("transaction_id")).alias("count_cust"))
customers_count.filter("count_cust > 1").show()

+-----------+----------+
|customer_id|count_cust|
+-----------+----------+
+-----------+----------+



In [16]:
transactions_df_enriched.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- price: double (nullable = true)
 |-- transaction_date: timestamp (nullable = true)
 |-- store_location: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- discount_applied: double (nullable = true)
 |-- transaction_year: integer (nullable = true)
 |-- transaction_month: integer (nullable = true)



In [9]:
#Lets cache the dataframe thats enriched and data handled for reuse

In [17]:
# Repartition by customer_id for better join performance
customers_df_enriched = customers_df_enriched.repartition("customer_id")
transactions_df_enriched = transactions_df_enriched.repartition("customer_id")

# Cache for reuse
customers_df_enriched.cache()
transactions_df_enriched.cache()


DataFrame[transaction_id: string, customer_id: string, product_name: string, product_category: string, quantity: double, price: double, transaction_date: timestamp, store_location: string, payment_method: string, discount_applied: double, transaction_year: int, transaction_month: int]

In [12]:
#Lets Explore Joins

In [18]:
#Inner Join

inner_join_df = transactions_df_enriched.join(customers_df_enriched, how="inner", on="customer_id")


In [19]:
#Left Outer Join

left_outer_join_df = transactions_df_enriched.join(customers_df_enriched, how="left", on="customer_id")

In [20]:
#Right Outer Join

right_outer_join_df = transactions_df_enriched.join(customers_df_enriched, how="right", on="customer_id")

In [21]:
#Full Outer Join

full_outer_join_df = transactions_df_enriched.join(customers_df_enriched, how="full", on="customer_id")

In [22]:
#Broadcast Join

broadcast_join_df =transactions_df_enriched.join(broadcast(customers_df_enriched), how="outer", on="customer_id")

In [18]:
#Common Aggregations & Transformations

In [23]:
#To Find Total Spend per customer, lets first create a amount column with price & quantity and then apply the discount percentage

#transactions_df_enriched = transactions_df_enriched.withColumn("amount", col("quantity") * col("price"))

#transactions_df_enriched = transactions_df_enriched.withColumn("final_amount", col("amount") - col("discount_applied"))


#We found NULL being reported for most final_amount - Lets Investigate

transactions_df_enriched.select("quantity", "price", "discount_applied")\
                        .where(col("quantity").isNull() | col("price").isNull() | col("discount_applied").isNull())\
                        .show()

#It is now investigated that in the dataset there is Quqntity value blank when price is present thus it cascades null to final_amount

+--------+-------+----------------+
|quantity|  price|discount_applied|
+--------+-------+----------------+
|     2.0|1425.13|            null|
|     2.0| 744.62|            null|
|     1.0| 746.84|            null|
|     1.0|1928.22|            null|
|     1.0| 125.76|            null|
|     1.0|   40.8|            null|
|     2.0|   62.1|            null|
|     1.0| 134.51|            null|
|     1.0|  207.4|            null|
|     1.0|  51.95|            null|
|     2.0| 459.22|            null|
|     2.0| 158.67|            null|
|     1.0|2097.37|            null|
|     1.0| 808.35|            null|
|     1.0|  27.58|            null|
|     1.0| 138.24|            null|
|     2.0| 721.28|            null|
|     1.0| 118.36|            null|
|     1.0|1298.82|            null|
|     1.0|  647.8|            null|
+--------+-------+----------------+
only showing top 20 rows



In [24]:
#It is now investigated that in the dataset there is Quqntity value blank when price is present thus it cascades null to final_amount
#Lets create safe columns without NUll before calculating Final Amount

#We tried coalesce but that was replacing all with 0.0 because quantity is blank and not null. So alrernate method below

from pyspark.sql.functions import col, when, lit

# Replace empty strings or nulls with 0 for quantity and price
transactions_df_enriched = transactions_df_enriched\
                        .withColumn("clean_quantity", when((col("quantity").isNull()) | (col("quantity") == ""), lit(0))\
                                    .otherwise(col("quantity").cast("int")))\
                        .withColumn("clean_price", when((col("price").isNull()) | (col("price") == ""), lit(0.0))\
                                    .otherwise(col("price").cast("double")))\
                        .withColumn("clean_discount", when((col("discount_applied").isNull()) | (col("discount_applied") == ""), lit(0))\
                                    .otherwise(col("discount_applied").cast("int")))\
                        .withColumn("amount", col("clean_quantity") * col("clean_price"))\
                        .withColumn("final_amount", col("amount") - col("clean_discount"))



In [25]:
#Now that we got the final columns - final_amount / amount we can drop the safe columns we created.

transactions_df_enriched = transactions_df_enriched.drop("clean_quantity","clean_price","clean_discount")

In [26]:
#To Find Total Spend per customer

total_spend_df = transactions_df_enriched.groupBy("customer_id").agg(sum("final_amount").alias("total_spend"))

In [27]:
#Average Spend Per Store

average_spend_df = transactions_df_enriched.groupBy("store_location").agg(avg("final_amount").alias("avg_spend"))

In [28]:
#Count of Transactions Per Payment Method

count_trans_paymentMethod = transactions_df_enriched.groupBy("payment_method").agg(count("transaction_id"))


In [29]:
#High-Value Transactions (Final Amount > 1000)

transactions_highvalue_df = transactions_df_enriched.filter("final_amount > 1000")

In [30]:
#Top N Customers by Spend

top_customer_df = total_spend_df.orderBy(col("total_spend").desc()).limit(10)



In [90]:
# Count of Transactions per City - this may not give accurate result

#trans_city_df = customers_df_enriched.join(transactions_df_enriched, on="customer_id", how = "left").groupBy("city").count()

In [31]:
# Count of Transactions per City

trans_city_df = customers_df_enriched.join(transactions_df_enriched, on="customer_id", how = "left").groupBy("city").agg(count(when(col("transaction_id").isNotNull(),1)).alias("transaction_count"))

In [32]:
#Average Spend per Month

avg_spend_month = transactions_df_enriched.groupBy("transaction_month").agg(avg("final_amount").alias("avg_spend_month"))

In [35]:
#Rename & Drop Columns
renamed_df = transactions_df_enriched.withColumnRenamed("total_amount" , "final_amount")

transactions_df_enriched = renamed_df.drop("total_amount")




In [36]:
transactions_df_enriched.show()

+--------------------+--------------------+------------------+--------------------+--------+-------+-------------------+---------------+--------------+----------------+----------------+-----------------+-----------------+-----------------+
|      transaction_id|         customer_id|      product_name|    product_category|quantity|  price|   transaction_date| store_location|payment_method|discount_applied|transaction_year|transaction_month|           amount|     final_amount|
+--------------------+--------------------+------------------+--------------------+--------+-------+-------------------+---------------+--------------+----------------+----------------+-----------------+-----------------+-----------------+
|bde856d7-22e6-441...|1280bddb-d9ed-4dc...|           Toaster|Small Kitchen App...|     1.0| 206.91|2023-01-07 00:00:00|    Seattle, WA|     Apple Pay|             0.0|            2023|                1|           206.91|           206.91|
|38c27959-ac74-4a2...|256c3dc8-04a9-41b.

In [37]:
#Adding Derived Columns (withColumn)

from pyspark.sql.functions import when

transactions_flagged_df = transactions_df_enriched.withColumn("high_value_flag",when(transactions_df_enriched["amount"] > 1000, "HIGH")\
                                                                .otherwise("NORMAL"))


In [38]:
#Select and Rename Columns

renamed_df = transactions_flagged_df.selectExpr("transaction_id as txn_id","customer_id","amount","high_value_flag")


In [95]:
renamed_df.show()

+--------------------+--------------------+-----------------+---------------+
|              txn_id|         customer_id|           amount|high_value_flag|
+--------------------+--------------------+-----------------+---------------+
|bde856d7-22e6-441...|1280bddb-d9ed-4dc...|           206.91|         NORMAL|
|38c27959-ac74-4a2...|256c3dc8-04a9-41b...|           859.13|         NORMAL|
|bb0009d5-b09f-4a1...|2fa8b3e5-af90-4be...|           254.09|         NORMAL|
|2cb4b02f-c804-45c...|2fe076bb-d401-427...|           268.68|         NORMAL|
|93fe10fd-1ba6-4cc...|377dacf5-92d0-496...|           1484.7|           HIGH|
|c5cc3025-81fd-4da...|43d172e3-4ee1-475...|           705.05|         NORMAL|
|073e8c29-b8b3-4d9...|455c05cc-3c5c-4f0...|          1759.76|           HIGH|
|e77fd735-4c34-4f2...|54e325e2-8499-44e...|6362.789999999999|           HIGH|
|58e89024-3035-431...|64afc1c8-2eaa-446...|           655.57|         NORMAL|
|87ba9c24-e11f-472...|81b29f5b-a070-45a...|            50.24|   

In [ ]:
# Repartition and Cache (optional for performance tuning)

In [41]:
sample_txns.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- transaction_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- price: double (nullable = true)
 |-- transaction_date: timestamp (nullable = true)
 |-- store_location: string (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- discount_applied: double (nullable = true)
 |-- transaction_year: integer (nullable = true)
 |-- transaction_month: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- final_amount: double (nullable = true)



In [42]:
from pyspark.sql.functions import lit, monotonically_increasing_id
from pyspark.sql import DataFrame

# 1. Pick a sample of customers to duplicate
sample_customers = transactions_df_enriched.select("customer_id").distinct().limit(5)




In [43]:
# 2. Join with transactions to get their original transactions
sample_txns = sample_customers.join(transactions_df_enriched, on="customer_id", how="inner")



In [45]:
# 3. Create new fake transactions by modifying some values
duplicated_txns = sample_txns.withColumn("transaction_id", monotonically_increasing_id()) \
    .withColumn("quantity", sample_txns["quantity"] + 1) \
    .withColumn("price", sample_txns["price"] * 1.2) \
    .withColumn("final_amount", (sample_txns["quantity"] + 1.0) * (sample_txns["price"] * 1.2))

AnalysisException: Resolved attribute(s) quantity#2126,price#2127 missing from customer_id#59,transaction_id#2193L,product_name#2124,product_category#2125,quantity#2208,price#2223,transaction_date#2128,store_location#2129,payment_method#2130,discount_applied#2131,transaction_year#102,transaction_month#114,amount#1199,final_amount#1702 in operator !Project [customer_id#59, transaction_id#2193L, product_name#2124, product_category#2125, quantity#2208, price#2223, transaction_date#2128, store_location#2129, payment_method#2130, discount_applied#2131, transaction_year#102, transaction_month#114, amount#1199, ((quantity#2126 + 1.0) * (price#2127 * 1.2)) AS final_amount#2238]. Attribute(s) with the same name appear in the operation: quantity,price. Please check if the right attribute(s) are used.;
!Project [customer_id#59, transaction_id#2193L, product_name#2124, product_category#2125, quantity#2208, price#2223, transaction_date#2128, store_location#2129, payment_method#2130, discount_applie

In [48]:
#WINDOW Functions..

#Rank by amount per customer

from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, rank, dense_rank, sum as _sum, lag, lead

window_spec_rank = Window.partitionBy("customer_id").orderBy(col("total_amount").desc())

ranked_products_df = transactions_df_enriched.withColumn("rank_in_customer",row_number().over(window_spec_rank))


AnalysisException: Column 'total_amount' does not exist. Did you mean one of the following? [final_amount, amount, store_location, product_name, quantity, transaction_id, customer_id, price, transaction_date, transaction_month, discount_applied, payment_method, product_category, transaction_year];
'Project [transaction_id#58, customer_id#59, product_name#60, product_category#61, quantity#62, price#63, transaction_date#64, store_location#65, payment_method#66, discount_applied#67, transaction_year#102, transaction_month#114, amount#1199, final_amount#1702, row_number() windowspecdefinition(customer_id#59, 'total_amount DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS rank_in_customer#2240]
+- Project [transaction_id#58, customer_id#59, product_name#60, product_category#61, quantity#62, price#63, transaction_date#64, store_location#65, payment_method#66, discount_applied#67, transaction_year#102, transaction_month#114, amount#1199, total_amount#13

In [102]:
transactions_df_enriched.select("customer_id","amount","total_amount").show()

+--------------------+-----------------+-----------------+
|         customer_id|           amount|     total_amount|
+--------------------+-----------------+-----------------+
|1280bddb-d9ed-4dc...|           206.91|           206.91|
|256c3dc8-04a9-41b...|           859.13|           839.13|
|2fa8b3e5-af90-4be...|           254.09|           254.09|
|2fe076bb-d401-427...|           268.68|           268.68|
|377dacf5-92d0-496...|           1484.7|           1484.7|
|43d172e3-4ee1-475...|           705.05|           705.05|
|455c05cc-3c5c-4f0...|          1759.76|          1754.76|
|54e325e2-8499-44e...|6362.789999999999|6362.789999999999|
|64afc1c8-2eaa-446...|           655.57|           625.57|
|81b29f5b-a070-45a...|            50.24|            50.24|
|861c66ad-3aec-4da...|          2448.24|          2448.24|
|8a185511-3dc2-469...|           1489.2|           1484.2|
|99ba027b-24e2-406...|            354.4|            339.4|
|bdb529e6-281d-4ab...|            67.78|            42.7

In [103]:
ranked_products_df.select("customer_id","total_amount","rank_in_customer").show()

+--------------------+-----------------+----------------+
|         customer_id|     total_amount|rank_in_customer|
+--------------------+-----------------+----------------+
|1280bddb-d9ed-4dc...|           206.91|               1|
|256c3dc8-04a9-41b...|           839.13|               1|
|2fa8b3e5-af90-4be...|           254.09|               1|
|2fe076bb-d401-427...|           268.68|               1|
|377dacf5-92d0-496...|           1484.7|               1|
|43d172e3-4ee1-475...|           705.05|               1|
|455c05cc-3c5c-4f0...|          1754.76|               1|
|54e325e2-8499-44e...|6362.789999999999|               1|
|64afc1c8-2eaa-446...|           625.57|               1|
|81b29f5b-a070-45a...|            50.24|               1|
|861c66ad-3aec-4da...|          2448.24|               1|
|8a185511-3dc2-469...|           1484.2|               1|
|99ba027b-24e2-406...|            339.4|               1|
|bdb529e6-281d-4ab...|            42.78|               1|
|dcdb6180-6f27

In [50]:
#Running total of final_amount per customer

window_spec_running = Window.partitionBy("customer_id").orderBy("transaction_date")

running_total_df = transactions_df_enriched.withColumn("running_total",_sum("final_amount").over(window_spec_running))


In [51]:
#Compare each transaction with previous

window_spec_lag = Window.partitionBy("customer_id").orderBy("transaction_date")

lag_lead_df = transactions_df_enriched.withColumn("prev_amount",lag("final_amount", 1).over(window_spec_lag))\
            .withColumn("next_amount",lead("final_amount", 1).over(window_spec_lag))


In [52]:
#Dense Rank: Cities by total spend

city_total_df = customers_df_enriched.join(transactions_df_enriched, "customer_id")

city_spend_df = city_total_df.groupBy("city").agg(
    _sum("final_amount").alias("total_spent")
)

window_spec_city = Window.orderBy(col("total_spent").desc())

city_ranked_df = city_spend_df.withColumn(
    "dense_rank", dense_rank().over(window_spec_city)
)


In [59]:
print("Before repartition:", transactions_df_enriched.rdd.getNumPartitions())

Before repartition: 200


In [58]:
transactions_df_repartitioned = transactions_df_enriched.repartition(10, "customer_id")

In [61]:
print("After repartition:", transactions_df_repartitioned.rdd.getNumPartitions())

After repartition: 10


In [62]:
"""Performance Tuning...!!!!!
Step 8: Repartition, Coalesce, and Caching
These are critical concepts when optimizing performance in PySpark — especially for large datasets.
8.1 Repartitioning
When to use?
Increase parallelism when working with larger datasets
Ensures better load balancing across Spark executors

# Check current number of partitions
print("Before repartition:", transactions_df_with_dupes.rdd.getNumPartitions())  --> 200

# Repartition to 10 partitions based on customer_id
transactions_df_repartitioned = transactions_df_with_dupes.repartition(10, "customer_id")

print("After repartition:", transactions_df_repartitioned.rdd.getNumPartitions())
✅ 8.2 Coalesce
When to use?
Reduce number of partitions

Especially useful before writing to disk or performing actions like collect()
# Coalesce into fewer partitions (e.g., 2) before saving
transactions_df_coalesced = transactions_df_repartitioned.coalesce(2)
✅ 8.3 Caching / Persisting
When to use?
If you're using a DataFrame multiple times in your pipeline

# Cache to memory
transactions_df_repartitioned.cache()
transactions_df_repartitioned.count()  # Trigger cache
or

from pyspark.storagelevel import StorageLevel

# Persist to memory and disk
transactions_df_repartitioned.persist(StorageLevel.MEMORY_AND_DISK)

"""

'Performance Tuning...!!!!!\nStep 8: Repartition, Coalesce, and Caching\nThese are critical concepts when optimizing performance in PySpark — especially for large datasets.\n8.1 Repartitioning\nWhen to use?\nIncrease parallelism when working with larger datasets\nEnsures better load balancing across Spark executors\n\n# Check current number of partitions\nprint("Before repartition:", transactions_df_with_dupes.rdd.getNumPartitions())  --> 200\n\n# Repartition to 10 partitions based on customer_id\ntransactions_df_repartitioned = transactions_df_with_dupes.repartition(10, "customer_id")\n\nprint("After repartition:", transactions_df_repartitioned.rdd.getNumPartitions())\n✅ 8.2 Coalesce\nWhen to use?\nReduce number of partitions\n\nEspecially useful before writing to disk or performing actions like collect()\n# Coalesce into fewer partitions (e.g., 2) before saving\ntransactions_df_coalesced = transactions_df_repartitioned.coalesce(2)\n✅ 8.3 Caching / Persisting\nWhen to use?\nIf you\'re

In [63]:
transactions_df_coalesced = transactions_df_repartitioned.coalesce(2)

In [64]:
print("After Coalesce:", transactions_df_coalesced.rdd.getNumPartitions())

After Coalesce: 2


In [65]:
#Save output as csv

transactions_df_repartitioned \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("gs://retail_pysparkdata//output/transactions")


In [66]:
#Save output as parque

transactions_df_repartitioned \
    .write \
    .mode("overwrite") \
    .parquet("gs://retail_pysparkdata//output/transactionss_parquet")
